# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a model

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [3]:
datafile_train="Property_train.csv"
datafile_test="Property_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [4]:
bd_train.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,...,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,...,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,...,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [5]:
# we are going to drop date column 'ListDate', you should extract components 
# from it and use them as features . create cyclic features from cyclic components 



In [6]:
target='Junk'

In [7]:
## look at the columns carefully and see what variables are stored as categorical but 
## need to be treated as numeric [PriceIndex8] . And some which are stored as numeric but are representing 
## categorical information [such as Zip]

bd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62035 entries, 0 to 62034
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Junk                   62035 non-null  int64 
 1   InteriorsStyle         62035 non-null  object
 2   PriceIndex8            62035 non-null  object
 3   ListDate               62035 non-null  object
 4   Material               62035 non-null  object
 5   PriceIndex9            62035 non-null  int64 
 6   Agency                 62035 non-null  object
 7   AreaIncomeType         62035 non-null  object
 8   EnvRating              62035 non-null  object
 9   PriceIndex7            62035 non-null  object
 10  ExpeditedListing       62035 non-null  int64 
 11  PriceIndex4            62035 non-null  object
 12  PriceIndex1            62035 non-null  object
 13  PriceIndex6            62035 non-null  object
 14  PRIMEUNIT              62035 non-null  object
 15  Channel            

In [8]:
bd_train.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,...,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,...,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,...,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [9]:
cat_to_numeric=['PriceIndex'+str(i) for i in range(1,10)]

In [11]:
bd_train['data']='train'
bd_test['data']='test'

all_data=pd.concat([bd_train,bd_test],axis=0,sort=False)

In [12]:
for col in cat_to_numeric:
    all_data[col]=pd.to_numeric(all_data[col],errors='coerce')
    all_data[col]=all_data[col].fillna(all_data.loc[all_data['data']=='train',col].median())

In [13]:
cat_cols=['InteriorsStyle', 'ListDate', 'Material', 'Agency', 'AreaIncomeType',
       'EnvRating', 'PRIMEUNIT', 'Channel', 'PlotType', 'Architecture',
       'Region', 'SubModel', 'Facade', 'State', 'RegionType','Zip']

In [14]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=1000].index[:-1]
    
    for cat in cats:
        name=col+'_'+str(cat)
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]
# you can try more lenient frequency cutoff , which will ofcourse 

In [15]:
all_data.shape

(72983, 125)

In [16]:
all_data.head()

,Junk,PriceIndex8,PriceIndex9,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,InsurancePremiumIndex,PriceIndex3,...,Zip_76040,Zip_77041,Zip_78754,Zip_80011,Zip_73108,Zip_28625,Zip_91752,Zip_78219,Zip_30272,Zip_32124
0,0.0,14674.0,8270,13143.0,0,14224.0,9217.0,10387.0,623,13108.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,4172.0,3890,3461.0,0,4404.0,2958.0,3400.0,1689,3695.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,8210.0,5900,6922.0,0,8952.0,4568.0,4862.0,2351,7730.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,7309.0,7460,6300.0,0,7460.0,5361.0,6305.0,1933,6290.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,9182.0,6400,8330.0,0,9846.0,5428.0,5718.0,482,8583.0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
x_train=all_data.drop([target,'data'],axis=1)[all_data['data']=='train']
y_train=all_data[target][all_data['data']=='train']
x_test=all_data.drop([target,'data'],axis=1)[all_data['data']=='test']

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [20]:
params={'penalty':['l1','l2'],'class_weight':['balanced',None],
        'C':np.linspace(0.01,100,100)}


In [21]:
model=LogisticRegression()

# tune more complex algos for improving performance

In [22]:
rs=RandomizedSearchCV(model,param_distributions=params,n_iter=10,
                      scoring='roc_auc',cv=10,n_jobs=-1,verbose=20)

In [23]:
rs.fit(x_train,y_train)

# this takes quite awhile to run on my machine , more complex algos 
# are going to take much more time , couple hrs may be for parameter tuning 
# do preliminary feature selection first a simple random forest and then do parameter tuning

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
                   param_distributions={'C': array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2....
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 8.283e+01, 8.384e+01,
       8.485e+01, 8.586e+01, 8.687e+01, 8.788e+01, 8.889e+01, 8.990e+01,
       9.091e+01, 9.192e+01, 9.293e+01, 9.394e+01, 9.495e+01, 9.596e+01,
       9.697e+01, 9.798e+01, 9.899e+01, 1.000e+02]),
                                        'class_weight': ['balanced', None],
                                        'penalty': ['l1', 'l2']},
                   scoring='roc_auc', verbose=20)

In [24]:
submissions=pd.DataFrame({'Junk':rs.predict_proba(x_test)[:,1]})

In [25]:
submissions.to_csv('submission.csv',index=False)